In [1]:
import basedosdados as bd
import pandas as pd

In [4]:
bd.list_datasets(filter_by='nomes')


dataset_id: 
	br_ibge_nomes_brasil 
-----------------------------------------------------------------------------------------------



In [8]:
bd.read_table(dataset_id='br_ibge_populacao', 
            table_id='municipios',
            billing_project_id='steam-snowfall-306819'
             )

Downloading: 100%|████████████████████████████████████████████████████████| 163248/163248 [00:05<00:00, 30229.89rows/s]


,id_municipio,ano,populacao
0,1100015,1991,31981.0
1,1100015,1992,34768.0
2,1100015,1993,37036.0
3,1100015,1994,39325.0
4,1100015,1995,41574.0
...,...,...,...
163243,5300108,2016,2977216.0
163244,5300108,2017,3039444.0
163245,5300108,2018,2974703.0
163246,5300108,2019,3015268.0
